## Load Python Modules


In [2]:
from bs4 import BeautifulSoup
import random
import re
import csv

## Authenticate with your GCP Project

In [5]:
from google.colab import auth
auth.authenticate_user()

# https://cloud.google.com/resource-manager/docs/creating-managing-projects
project_id = 'replace-with-your-project-id'
!gcloud config set project {project_id}

Updated property [core/project].


## Import Data from BigQuery

In [7]:
%%bigquery df --project replace-with-your-project-id
SELECT 
    Q.id,
    Q.title,
    Q.body AS question_body,
    Q.answer_count,
    Q.comment_count,
    Q.favorite_count,
    Q.score,
    Q.tags,
    Q.view_count,
    A.body AS answer_body
FROM `bigquery-public-data.stackoverflow.posts_questions` AS Q
INNER JOIN `bigquery-public-data.stackoverflow.posts_answers` AS A
    ON Q.accepted_answer_id = A.id
WHERE (Q.tags LIKE '%google%' and Q.tags LIKE '%cloud%') or Q.tags LIKE '%dialogflow%'

## Format training data for AutoML Natural Language

In [9]:
train_split = ["TRAIN"] * round(len(df)*.7) + ["VALIDATION"] * round(len(df)*.2) + ["TEST"] * round(len(df)*.1)
random.shuffle(train_split)
df["train_split"] = train_split

csv_file = open("train_data.csv", "w")
writer = csv.writer(csv_file)

for index, row in df.iterrows():
    soup = BeautifulSoup(df.question_body[index])
    res = df.title[index] + " " + soup.text
    text = res.replace('\n',' ').replace("\'", "'").replace('"',"'")
    
    labels = re.findall("(?:google|dialogflow)(?:-[a-z]+){1,4}", df.tags[index].replace("|"," "))
    
    writer.writerow([df["train_split"][index]] + ['"' + text + '"'] + labels)
    #print([df["train_split"][index]] + ['"' + text + '"'] + labels)

csv_file.close()